# Import Required Packages

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [4]:
#training controls
batch_size = 32
epochs = 20
training_size = 0.8
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [5]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [6]:
# opening the dataset
dataset = csv.reader(open("MovieGenre.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['\ufeffimdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [7]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("SampleMoviePosters.zip", 'r')
files.extractall()
files.close()

In [8]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('SampleMoviePosters/*.jpg')  

In [9]:
len(flist)

997

In [10]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [11]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
40103,83168,http://www.imdb.com/title/tt83168,Tanya's Island (1980),4.3,Drama,https://images-na.ssl-images-amazon.com/images...
40104,82875,http://www.imdb.com/title/tt82875,Pacific Banana (1981),4.7,Comedy,https://images-na.ssl-images-amazon.com/images...
40105,815258,http://www.imdb.com/title/tt815258,Werewolf in a Womens Prison (2006),4.5,Horror,https://images-na.ssl-images-amazon.com/images...
40106,79142,http://www.imdb.com/title/tt79142,Xiao zi ming da (1979),6.5,Action|Comedy,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [12]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [13]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
15175,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
22276,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
22670,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13802,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
17534,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19630,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14532,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
23196,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
24453,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,24252,http://www.imdb.com/title/tt24252,Liebelei (1933),7.7,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,25316,http://www.imdb.com/title/tt25316,It Happened One Night (1934),8.2,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,25164,http://www.imdb.com/title/tt25164,The Gay Divorcee (1934),7.6,Comedy|Musical|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,17350,http://www.imdb.com/title/tt17350,The Scarlet Letter (1926),7.8,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,25586,http://www.imdb.com/title/tt25586,Of Human Bondage (1934),7.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1013,23797,http://www.imdb.com/title/tt23797,Betty Boop's Big Boss (1933),6.6,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1014,22670,http://www.imdb.com/title/tt22670,Betty Boop's Museum (1932),6.9,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1015,19640,http://www.imdb.com/title/tt19640,Angora Love (1929),7.0,Comedy|Short,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1016,24676,http://www.imdb.com/title/tt24676,TÌ«kyÌ« no onna (1933),7.2,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [17]:
#setting the length of training data
length=int(len(flist)*training_size)
length

797

In [18]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [19]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [20]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [21]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (797, 3, 268, 182)
797 train samples
221 test samples


In [22]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

# Model 1: Resnet50

In [22]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

In [23]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
from statistics import mean

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1 or preds[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1 or preds[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/797 (0%)]	Loss: 0.923549	Accuracy: 9.31%
Train Epoch: 0 [32/797 (4%)]	Loss: 0.917715	Accuracy: 4.11%
Train Epoch: 0 [64/797 (8%)]	Loss: 0.903031	Accuracy: 4.48%
Train Epoch: 0 [96/797 (12%)]	Loss: 0.884587	Accuracy: 5.26%
Train Epoch: 0 [128/797 (16%)]	Loss: 0.872908	Accuracy: 2.97%
Train Epoch: 0 [160/797 (20%)]	Loss: 0.862817	Accuracy: 1.04%
Train Epoch: 0 [192/797 (24%)]	Loss: 0.849016	Accuracy: 3.39%
Train Epoch: 0 [224/797 (28%)]	Loss: 0.842407	Accuracy: 3.65%
Train Epoch: 0 [256/797 (32%)]	Loss: 0.825192	Accuracy: 3.12%
Train Epoch: 0 [288/797 (36%)]	Loss: 0.818679	Accuracy: 1.56%
Train Epoch: 0 [320/797 (40%)]	Loss: 0.808601	Accuracy: 5.73%
Train Epoch: 0 [352/797 (44%)]	Loss: 0.797320	Accuracy: 2.60%
Train Epoch: 0 [384/797 (48%)]	Loss: 0.791466	Accuracy: 7.29%
Train Epoch: 0 [416/797 (52%)]	Loss: 0.782009	Accuracy: 9.38%
Train Epoch: 0 [448/797 (56%)]	Loss: 0.775697	Accuracy: 7.81%
Train Epoch: 0 [480/797 (60%)]	Loss: 0.769108	Accuracy: 8.85%
Train Epoch: 0 [

<ipython-input-24-0181edc6d50c>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.7265 
Accuracy: 19.54%
Train Epoch: 1 [0/797 (0%)]	Loss: 0.723777	Accuracy: 17.19%
Train Epoch: 1 [32/797 (4%)]	Loss: 0.720568	Accuracy: 18.75%
Train Epoch: 1 [64/797 (8%)]	Loss: 0.717851	Accuracy: 13.02%
Train Epoch: 1 [96/797 (12%)]	Loss: 0.713420	Accuracy: 15.62%
Train Epoch: 1 [128/797 (16%)]	Loss: 0.714994	Accuracy: 18.23%
Train Epoch: 1 [160/797 (20%)]	Loss: 0.711575	Accuracy: 23.96%
Train Epoch: 1 [192/797 (24%)]	Loss: 0.709704	Accuracy: 14.06%
Train Epoch: 1 [224/797 (28%)]	Loss: 0.708644	Accuracy: 19.27%
Train Epoch: 1 [256/797 (32%)]	Loss: 0.705935	Accuracy: 19.27%
Train Epoch: 1 [288/797 (36%)]	Loss: 0.705236	Accuracy: 19.79%
Train Epoch: 1 [320/797 (40%)]	Loss: 0.704900	Accuracy: 25.00%
Train Epoch: 1 [352/797 (44%)]	Loss: 0.701538	Accuracy: 14.58%
Train Epoch: 1 [384/797 (48%)]	Loss: 0.704748	Accuracy: 24.48%
Train Epoch: 1 [416/797 (52%)]	Loss: 0.701353	Accuracy: 28.65%
Train Epoch: 1 [448/797 (56%)]	Loss: 0.701851	Accuracy: 29.69%
Train Epo

# Model 2: DenseNet201

In [72]:
# DenseNet201

import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, 23)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [74]:
from statistics import mean

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1:
                    denom += 1
                    if int(preds[i][j]) == int(target[i][j]):
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1:
                    denom += 1
                    if int(preds[n][m]) == int(target[n][m]):
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/797 (0%)]	Loss: 0.421658	Accuracy: 11.98%
Train Epoch: 0 [32/797 (4%)]	Loss: 0.409568	Accuracy: 18.75%
Train Epoch: 0 [64/797 (8%)]	Loss: 0.390924	Accuracy: 23.44%
Train Epoch: 0 [96/797 (12%)]	Loss: 0.382522	Accuracy: 21.35%
Train Epoch: 0 [128/797 (16%)]	Loss: 0.368344	Accuracy: 22.40%
Train Epoch: 0 [160/797 (20%)]	Loss: 0.372027	Accuracy: 16.67%
Train Epoch: 0 [192/797 (24%)]	Loss: 0.346041	Accuracy: 23.44%
Train Epoch: 0 [224/797 (28%)]	Loss: 0.325197	Accuracy: 25.52%
Train Epoch: 0 [256/797 (32%)]	Loss: 0.330343	Accuracy: 27.08%
Train Epoch: 0 [288/797 (36%)]	Loss: 0.322869	Accuracy: 19.79%
Train Epoch: 0 [320/797 (40%)]	Loss: 0.313639	Accuracy: 26.56%
Train Epoch: 0 [352/797 (44%)]	Loss: 0.307575	Accuracy: 20.83%
Train Epoch: 0 [384/797 (48%)]	Loss: 0.288649	Accuracy: 21.35%
Train Epoch: 0 [416/797 (52%)]	Loss: 0.315079	Accuracy: 22.40%
Train Epoch: 0 [448/797 (56%)]	Loss: 0.286303	Accuracy: 23.96%
Train Epoch: 0 [480/797 (60%)]	Loss: 0.302496	Accuracy: 13.54%


<ipython-input-74-d42b1c217955>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.2749 
Accuracy: 32.76%
Train Epoch: 1 [0/797 (0%)]	Loss: 0.209011	Accuracy: 25.00%
Train Epoch: 1 [32/797 (4%)]	Loss: 0.227726	Accuracy: 19.79%
Train Epoch: 1 [64/797 (8%)]	Loss: 0.202292	Accuracy: 36.98%
Train Epoch: 1 [96/797 (12%)]	Loss: 0.203805	Accuracy: 26.04%
Train Epoch: 1 [128/797 (16%)]	Loss: 0.194427	Accuracy: 33.85%
Train Epoch: 1 [160/797 (20%)]	Loss: 0.197576	Accuracy: 31.77%
Train Epoch: 1 [192/797 (24%)]	Loss: 0.191081	Accuracy: 30.21%
Train Epoch: 1 [224/797 (28%)]	Loss: 0.199480	Accuracy: 25.52%
Train Epoch: 1 [256/797 (32%)]	Loss: 0.190075	Accuracy: 34.38%
Train Epoch: 1 [288/797 (36%)]	Loss: 0.193137	Accuracy: 35.94%
Train Epoch: 1 [320/797 (40%)]	Loss: 0.194417	Accuracy: 26.56%
Train Epoch: 1 [352/797 (44%)]	Loss: 0.203427	Accuracy: 20.83%
Train Epoch: 1 [384/797 (48%)]	Loss: 0.181481	Accuracy: 21.35%
Train Epoch: 1 [416/797 (52%)]	Loss: 0.184194	Accuracy: 27.08%
Train Epoch: 1 [448/797 (56%)]	Loss: 0.186116	Accuracy: 27.08%
Train Epo

# Model 3: ResNet152

In [23]:
# Resnet152 model
from torchvision import models

class ResNet152(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

In [24]:
model3 = ResNet152()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /home/rjw8ng/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 148MB/s] 


In [26]:
from statistics import mean

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/797 (0%)]	Loss: 0.883684	Accuracy: 23.96%
Train Epoch: 0 [32/797 (4%)]	Loss: 0.871886	Accuracy: 22.40%
Train Epoch: 0 [64/797 (8%)]	Loss: 0.863497	Accuracy: 14.06%
Train Epoch: 0 [96/797 (12%)]	Loss: 0.849553	Accuracy: 8.85%
Train Epoch: 0 [128/797 (16%)]	Loss: 0.837823	Accuracy: 11.98%
Train Epoch: 0 [160/797 (20%)]	Loss: 0.826048	Accuracy: 8.85%
Train Epoch: 0 [192/797 (24%)]	Loss: 0.811757	Accuracy: 12.50%
Train Epoch: 0 [224/797 (28%)]	Loss: 0.806413	Accuracy: 12.50%
Train Epoch: 0 [256/797 (32%)]	Loss: 0.797472	Accuracy: 18.23%
Train Epoch: 0 [288/797 (36%)]	Loss: 0.786766	Accuracy: 9.38%
Train Epoch: 0 [320/797 (40%)]	Loss: 0.778789	Accuracy: 9.38%
Train Epoch: 0 [352/797 (44%)]	Loss: 0.772812	Accuracy: 16.15%
Train Epoch: 0 [384/797 (48%)]	Loss: 0.766647	Accuracy: 9.90%
Train Epoch: 0 [416/797 (52%)]	Loss: 0.756628	Accuracy: 14.58%
Train Epoch: 0 [448/797 (56%)]	Loss: 0.753020	Accuracy: 12.50%
Train Epoch: 0 [480/797 (60%)]	Loss: 0.743401	Accuracy: 12.50%
Train

<ipython-input-26-3aabfdccf29b>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.7122 
Accuracy: 1.72%
Train Epoch: 1 [0/797 (0%)]	Loss: 0.710102	Accuracy: 19.27%
Train Epoch: 1 [32/797 (4%)]	Loss: 0.708853	Accuracy: 19.27%
Train Epoch: 1 [64/797 (8%)]	Loss: 0.708464	Accuracy: 22.92%
Train Epoch: 1 [96/797 (12%)]	Loss: 0.705903	Accuracy: 19.27%
Train Epoch: 1 [128/797 (16%)]	Loss: 0.703999	Accuracy: 24.48%
Train Epoch: 1 [160/797 (20%)]	Loss: 0.701561	Accuracy: 22.40%
Train Epoch: 1 [192/797 (24%)]	Loss: 0.703255	Accuracy: 19.27%
Train Epoch: 1 [224/797 (28%)]	Loss: 0.700129	Accuracy: 25.00%
Train Epoch: 1 [256/797 (32%)]	Loss: 0.700992	Accuracy: 22.92%
Train Epoch: 1 [288/797 (36%)]	Loss: 0.700694	Accuracy: 25.52%
Train Epoch: 1 [320/797 (40%)]	Loss: 0.697252	Accuracy: 23.96%
Train Epoch: 1 [352/797 (44%)]	Loss: 0.695968	Accuracy: 27.08%
Train Epoch: 1 [384/797 (48%)]	Loss: 0.698793	Accuracy: 19.27%
Train Epoch: 1 [416/797 (52%)]	Loss: 0.694777	Accuracy: 22.92%
Train Epoch: 1 [448/797 (56%)]	Loss: 0.694376	Accuracy: 23.96%
Train Epoc